In [53]:
import pandas as pd
import numpy as np
import time
import pulp

In [54]:
# --- 1. Arquivos e Time ---
ARQUIVO_DADOS_PRINCIPAL = "player-data-full.csv"
ARQUIVO_DADOS_CUSTOM = "flamengo_custom_players.csv"
TIME_ESCOLHIDO = "Flamengo"
LIGA_TIME = "Premier League"
NUM_JANELAS = 4

# --- 2. Parâmetros Financeiros ---
TRANSFER_BUDGET_INICIAL = 15000000  # Orçamento inicial
PERCENTUAL_FOLGA_SALARIAL = 0.2   # Teto salarial = (Salário Atual * (1 + %))
VALOR_MAX_JOGADOR = 80000000     # Teto de valor para poda (evita Mbappé)

# --- 3. Parâmetros de Poda (Performance) ---
# Quão "profundo" o modelo deve olhar no mercado.
K_MERCADO_POR_POS = 200 # Top K jogadores por posição
K_PARES_POR_JOGADOR = 12  # Top K parceiros táticos para cada jogador do elenco

# --- 4. Filosofia (Pesos da Função Objetivo) ---
w_qualidade = 0.6   # Foco no "Vencer Agora" (Overall)
w_potencial = 0.3   # Foco em "Construir o Futuro" (Growth)
w_fisico = 0.1      # Foco em "Elenco de Ferro" (Physical)
# (Soma deve ser 1.0)

# --- 5. Parâmetros de Química (Estoque S) ---
DECAY_QUIMICA = 0.7        # 0<=decay<1. (0.7 = 30% de perda por janela)
GAIN_TITULARES = 1.0       # Ganho por janela titular
BONUS_ENTROSAMENTO = 250.0  # Multiplicador do "Estoque S" na função objetivo

# --- 6. Parâmetros do Solver ---
GAP_RELATIVO = 0.01        # 0.01 = Parar quando a solução for 1% ótima
LIMITE_TEMPO = 1800         # 900 segundos = 15 minutos

In [55]:
# --- [CARREGAMENTO E LIMPEZA DOS DADOS PRINCIPAIS] ---

print(f"Carregando {ARQUIVO_DADOS_PRINCIPAL}...")
try:
    player_data = pd.read_csv(ARQUIVO_DADOS_PRINCIPAL)
except FileNotFoundError:
    print(f"Erro: Arquivo {ARQUIVO_DADOS_PRINCIPAL} não encontrado.")
    # (Adicione uma parada ou lógica de erro aqui)

# Define as colunas que realmente usaremos
cols_necessarias = [
    "name", "dob", "country_name", "positions", "overall_rating", "potential", 
    "value", "wage", "club_name", "club_league_name", 
    "acceleration", "agility", "strength", "stamina"
]
df = player_data[cols_necessarias].copy()

Carregando player-data-full.csv...


/var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/ipykernel_73009/3848098436.py:5: DtypeWarning: Columns (15,16,34,35,36,37,39,73) have mixed types. Specify dtype option on import or set low_memory=False.
  player_data = pd.read_csv(ARQUIVO_DADOS_PRINCIPAL)


# Funções de pré-processamento

In [56]:
# --- [DEFINIÇÃO: FUNÇÕES DE PRÉ-PROCESSAMENTO] ---

def converter_valor_monetario(valor_str):
    """Converte strings como '€10.5M' ou '€200K' para float."""
    if isinstance(valor_str, str):
        valor_str = valor_str.replace('€', '').strip()
        if 'M' in valor_str:
            return float(valor_str.replace('M', '')) * 1000000
        elif 'K' in valor_str:
            return float(valor_str.replace('K', '')) * 1000
    try:
        return float(valor_str)
    except (ValueError, TypeError):
        return 0.0 # Retorna 0 se for NaN ou string inválida

def processar_dataframe(df):
    """Aplica todas as transformações de limpeza e engenharia de features."""
    print("Iniciando pré-processamento do DataFrame...")
    
    # 1. Valores Monetários
    df['value_eur'] = df['value'].apply(converter_valor_monetario)
    df['wage_eur'] = df['wage'].apply(converter_valor_monetario)
    
    # 2. Idade
    df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
    ano_atual = pd.to_datetime('now').year
    df['age'] = ano_atual - df['dob'].dt.year
    # Lida com NaNs em 'age' (se 'dob' for inválido)
    df['age'] = df['age'].fillna(df['age'].median()) 
    
    # 3. Físico (Média de 4 atributos)
    df['physical'] = ((df['stamina'] + df['strength'] + 
                       df['acceleration'] + df['agility']) / 4).round(0)
    
    # 4. Potencial de Crescimento
    df['growth_potential'] = df['potential'] - df['overall_rating']
    
    # 5. Posições
    df['main_position'] = df['positions'].str.split(',').str[0].str.strip()
    df['sec_positions'] = df['positions'].str.split(',').str[1:].str.join(', ').str.strip()
    df['versatility'] = df['positions'].str.split(',').str.len()
    
    # 6. Limpeza de Colunas
    cols_finais = [
        "name", "age", "country_name", "main_position", "sec_positions", 
        "overall_rating", "potential", "growth_potential", "value", "wage", 
        "club_name", "club_league_name", "physical", "versatility", 
        "value_eur", "wage_eur"
    ]
    # Remove colunas duplicadas (como 'country_name_y' se existisse)
    df = df.loc[:, ~df.columns.duplicated()]
    # Garante que todas as colunas necessárias existam, mesmo que vazias
    for col in cols_finais:
        if col not in df.columns:
            df[col] = np.nan
            
    df = df[cols_finais]
    
    # 7. Lida com valores nulos críticos
    df['wage_eur'] = df['wage_eur'].fillna(0)
    df['value_eur'] = df['value_eur'].fillna(0)
    
    print("Pré-processamento concluído.")
    return df

# Executa o processamento
df = processar_dataframe(df)
df.head()

Iniciando pré-processamento do DataFrame...
Pré-processamento concluído.


,name,age,country_name,main_position,sec_positions,overall_rating,potential,growth_potential,value,wage,club_name,club_league_name,physical,versatility,value_eur,wage_eur
0,Erling Haaland,25,Norway,ST,,91,94,3,€185M,€340K,Manchester City,Premier League,82.0,1,185000000.0,340000.0
1,Kylian Mbappé,27,France,ST,LW,91,94,3,€181.5M,€230K,Paris Saint Germain,Ligue 1,89.0,2,181500000.0,230000.0
2,Kevin De Bruyne,34,Belgium,CM,CAM,91,91,0,€103M,€350K,Manchester City,Premier League,78.0,2,103000000.0,350000.0
3,Rodri,29,Spain,CDM,CM,90,91,1,€122.5M,€260K,Manchester City,Premier League,74.0,2,122500000.0,260000.0
4,Harry Kane,32,England,ST,,90,90,0,€119.5M,€170K,FC Bayern München,Bundesliga,74.0,1,119500000.0,170000.0


In [57]:
# --- [MONTAGEM DO ELENCO INICIAL] ---

# 1. Limpa qualquer time com o nome escolhido (para evitar duplicatas)
print(f"Limpando dados antigos do {TIME_ESCOLHIDO}...")
df = df[df['club_name'] != TIME_ESCOLHIDO].copy()

# 2. Transfere os 7 jogadores "Legado"
print(f"Transferindo 7 jogadores 'Legado' para o {TIME_ESCOLHIDO}...")
jogadores_para_mudar = [
    {'name': 'Samuel Lino', 'club_name': None},
    {'name': 'Saúl', 'club_name': "Atlético Madrid"},
    {'name': 'Jorginho', 'club_name': None},
    {'name': 'Emerson Royal', 'club_name': None},
    {'name': 'Michael', 'club_name': None},
    {'name': 'Danilo', 'club_name': 'Juventus'},
    {'name': 'Alex Sandro', 'club_name': 'Juventus'}
]

for jogador in jogadores_para_mudar:
    nome_jogador = jogador['name']
    clube_original = jogador['club_name']
    condicao = (df['name'] == nome_jogador)
    if clube_original:
        condicao &= (df['club_name'] == clube_original)
    df.loc[condicao, ['club_name', 'club_league_name']] = [TIME_ESCOLHIDO, LIGA_TIME]

# 3. Carrega os 21 jogadores "Customizados" do arquivo CSV
print(f"Carregando jogadores customizados de {ARQUIVO_DADOS_CUSTOM}...")
try:
    df_novos_jogadores = pd.read_csv(ARQUIVO_DADOS_CUSTOM)
    
    # --- [INÍCIO DA CORREÇÃO] ---
    # Define o clube e a liga para os jogadores customizados,
    # pois o CSV não contém essa informação.
    df_novos_jogadores['club_name'] = TIME_ESCOLHIDO
    df_novos_jogadores['club_league_name'] = LIGA_TIME
    # --- [FIM DA CORREÇÃO] ---

    # Calcula 'growth_potential' e 'versatility' que não estão no CSV
    df_novos_jogadores['growth_potential'] = df_novos_jogadores['potential'] - df_novos_jogadores['overall_rating']
    df_novos_jogadores['versatility'] = 1 + df_novos_jogadores['sec_positions'].str.count(',').fillna(0)
    
    # 4. Concatena tudo
    df = pd.concat([df, df_novos_jogadores], ignore_index=True)
    
    print(f"Elenco customizado de {len(df_novos_jogadores)} jogadores adicionado.")
    print("\n--- Verificação do Elenco Inicial Completo ---")
    # Este display agora mostrará todos os 28 jogadores
    display(df[df['club_name'] == TIME_ESCOLHIDO])

except FileNotFoundError:
    print(f"Erro: Arquivo {ARQUIVO_DADOS_CUSTOM} não encontrado.")
    print("O elenco inicial consistirá apenas nos 7 jogadores 'Legado'.")
except Exception as e:
    print(f"Erro ao processar o CSV customizado: {e}")

Limpando dados antigos do Flamengo...
Transferindo 7 jogadores 'Legado' para o Flamengo...
Carregando jogadores customizados de flamengo_custom_players.csv...
Elenco customizado de 21 jogadores adicionado.

--- Verificação do Elenco Inicial Completo ---


,name,age,country_name,main_position,sec_positions,overall_rating,potential,growth_potential,value,wage,club_name,club_league_name,physical,versatility,value_eur,wage_eur
177,Jorginho,34,Italy,CDM,CM,83,83,0,€25.5M,€125K,Flamengo,Premier League,72.0,2.0,25500000.0,125000.0
337,Saúl,31,NaN,CM,,81,81,0,€25.5M,€65K,Flamengo,Premier League,72.0,1.0,25500000.0,65000.0
355,Danilo,34,NaN,CB,RB,81,81,0,€17.5M,€100K,Flamengo,Premier League,72.0,2.0,17500000.0,100000.0
509,Samuel Lino,26,NaN,LM,LWB,79,84,5,€26.5M,€48K,Flamengo,Premier League,78.0,2.0,26500000.0,48000.0
895,Emerson Royal,26,NaN,RB,CB,77,79,2,€12.5M,€56K,Flamengo,Premier League,74.0,2.0,12500000.0,56000.0
1087,Alex Sandro,34,NaN,CB,LB,77,77,0,€7M,€77K,Flamengo,Premier League,73.0,2.0,7000000.0,77000.0
1195,Michael,29,NaN,RM,LM,76,76,0,€8M,€36K,Flamengo,Premier League,78.0,2.0,8000000.0,36000.0
18311,Agustín Rossi,28,Argentina,GK,NaN,79,79,0,€12.5M,€29K,Flamengo,Premier League,74.0,1.0,12500000.0,29000.0
18312,Guillermo Varela,31,Uruguay,RB,LB,76,76,0,€5.5M,€35K,Flamengo,Premier League,73.0,1.0,5500000.0,35000.0
18313,Léo Ortiz,28,Brazil,CB,CDM,80,80,0,€18M,€47K,Flamengo,Premier League,77.0,1.0,18000000.0,47000.0


In [58]:
# --- [DEFINIÇÃO: FUNÇÕES DE SIMULAÇÃO DE EVOLUÇÃO] ---
# (Lógica da sua Célula [9] original, com comentários)

def calcular_mudanca_anual_ovr_suavizada(age, current_ovr, potential):
    """Calcula a mudança de OVR base (crescimento + declínio) para um ano."""
    
    # 1. Crescimento (só ocorre se OVR < Potencial)
    if current_ovr >= potential:
        growth = 0
    else:
        # Fases de crescimento
        if age < 22:
            growth = np.random.uniform(1.5, 4)
        elif age < 27:
            growth = np.random.uniform(1, 3)
        elif age < 30:
            growth = np.random.uniform(0, 1)
        else:
            growth = 0 # Para de crescer por idade após os 30

    # 2. Declínio (só ocorre após os 30)
    if age < 30:
        decline = 0
    elif age < 33:
        decline = np.random.uniform(-1, 0)
    elif age < 36:
        decline = np.random.uniform(-2, -1)
    else:
        decline = np.random.uniform(-4, -2)

    return growth + decline

def evoluir_valor_uma_janela(jogador_stats_anterior, novas_stats):
    """Calcula o NOVO valor de forma INCREMENTAL, baseado no valor ANTERIOR."""
    valor_anterior = jogador_stats_anterior['value_eur']
    
    # 1. Mudança por OVR
    ovr_change = novas_stats['overall_rating'] - jogador_stats_anterior['overall_rating']
    mult_ovr = 1.0 + (ovr_change * np.random.uniform(0.08, 0.12))
    
    # 2. Mudança por Idade (só se aplica no verão, quando a idade muda)
    mult_idade = 1.0
    age_change = novas_stats['age'] - jogador_stats_anterior['age']
    
    if age_change > 0: # Idade mudou (janela de verão)
        age = novas_stats['age']
        if age < 29:
            mult_idade = 1.05 # Valorização
        elif age < 32:
            mult_idade = 0.93 # Declínio suave
        elif age < 35:
            mult_idade = 0.88 # Declínio
        else:
            mult_idade = 0.82 # Declínio acentuado
            
    # 3. Bônus de Especulação (para potencial de crescimento)
    mult_potencial = 1.0
    if novas_stats['growth_potential'] > 0:
        mult_potencial = 1.0 + (novas_stats['growth_potential'] * 0.015) 
        
    fator_aleatorio = np.random.uniform(0.98, 1.02)
    
    novo_valor = valor_anterior * mult_ovr * mult_idade * mult_potencial * fator_aleatorio
    
    # Garante um valor mínimo para jogadores de elite
    if novas_stats['overall_rating'] > 80 and novo_valor < 1000000:
        return max(novo_valor, 1000000)
        
    return round(novo_valor, -3) # Arredonda para a casa dos milhar


def evoluir_jogador_uma_janela(jogador_stats, t):
    """Função principal que evolui um jogador por uma janela (6 meses)."""
    novas_stats = jogador_stats.copy()
    
    # 1. Atualiza a Idade (só no verão, t=2, t=4...)
    if t > 0 and t % 2 == 0:
        novas_stats['age'] = jogador_stats['age'] + 1
    
    # 2. Evolui Overall e Físico
    # (Mudança anual é dividida por 2, pois a janela é de 6 meses)
    mudanca_anual_ovr = calcular_mudanca_anual_ovr_suavizada(
        novas_stats['age'], novas_stats['overall_rating'], novas_stats['potential']
    )
    mudanca_anual_fisico = 0
    if novas_stats['age'] > 29:
        mudanca_anual_fisico = np.random.uniform(-2, 0)
    
    # Fator aleatório de "boa/má forma" na janela
    fator_aleatorio_forma = np.random.normal(0, 0.5) 
    
    mudanca_ovr_janela = (mudanca_anual_ovr / 2) + fator_aleatorio_forma
    mudanca_fisico_janela = mudanca_anual_fisico / 2
    
    # Garante que o OVR não ultrapasse o potencial
    gap_potencial = novas_stats['potential'] - novas_stats['overall_rating']
    if mudanca_ovr_janela > 0 and mudanca_ovr_janela > gap_potencial:
        mudanca_ovr_janela = max(0, gap_potencial)
    
    novas_stats['overall_rating'] = int(round(novas_stats['overall_rating'] + mudanca_ovr_janela))
    novas_stats['physical'] = int(round(novas_stats['physical'] + mudanca_fisico_janela))
    
    if novas_stats['overall_rating'] > novas_stats['potential']:
        novas_stats['overall_rating'] = novas_stats['potential']
        
    novas_stats['growth_potential'] = novas_stats['potential'] - novas_stats['overall_rating']
    
    # 4. Evolui o Valor
    novas_stats['value_eur'] = evoluir_valor_uma_janela(jogador_stats, novas_stats)
    
    return novas_stats

In [59]:
# --- [DEFINIÇÃO: LÓGICA DE TÁTICA E QUÍMICA] ---

# 1. Dicionário de Mapeamento de Posições
# (Converte posições do FIFA para posições agregadas do modelo)
mapa_posicoes = {
    'GK': 'GK', 'CB': 'CB', 'LB': 'LFB', 'RB': 'RFB', 'LWB': 'LFB',
    'RWB': 'RFB', 'CDM': 'MC', 'CM': 'MC', 'CAM': 'MC', 'LW': 'LWG',
    'RW': 'RWG', 'LM': 'LWG', 'RM': 'RWG', 'ST': 'ST', 'CF': 'ST'
}

# 2. Requisitos de Posição (Profundidade do Elenco)
# (O elenco final deve ter no mínimo X jogadores para cada posição agregada)
requisitos_posicao = {
    'GK': 3, 'CB': 4, 'LFB': 2, 'RFB': 2, 'MC': 7, 'LWG': 2, 'RWG': 2, 'ST': 3
}

# 3. Formação Titular (4-3-3)
# (Define a escalação titular que o modelo deve preencher)
formacao_titular = {
    'GK': 1, 'CB': 2, 'LFB': 1, 'RFB': 1, 'MC': 3, 'LWG': 1, 'RWG': 1, 'ST': 1
}
NUM_TITULARES = sum(formacao_titular.values()) # = 11

# 4. Pares Táticos (Quais posições "conversam" para química)
pares_taticos_set = {
    ('GK', 'CB'), ('CB', 'CB'), ('CB', 'LFB'), ('CB', 'RFB'), ('LFB', 'RFB'),
    ('LFB', 'MC'), ('RFB', 'MC'), ('CB', 'MC'), ('LFB', 'LWG'), ('RFB', 'RWG'),
    ('MC', 'MC'), ('MC', 'LWG'), ('MC', 'RWG'), ('MC', 'ST'),
    ('LWG', 'RWG'), ('LWG', 'ST'), ('RWG', 'ST'), ('ST', 'ST')
}
# Cria um conjunto simétrico (ex: (CB, GK) é o mesmo que (GK, CB))
pares_taticos = set(pares_taticos_set)
for (a, b) in pares_taticos_set:
    pares_taticos.add((b, a))

print(f"Definições de Tática e Química carregadas. {len(pares_taticos)} pares táticos definidos.")

Definições de Tática e Química carregadas. 33 pares táticos definidos.


In [60]:
# --- [PIPELINE ETAPA 1: PODA DO MERCADO] ---
print("Iniciando Poda 1: Filtrando mercado (Top-K por Posição)...")

# 1. Definir Elenco Atual e Mercado Completo
mercado_completo = df[df['club_name'] != TIME_ESCOLHIDO].copy()
elenco_atual = df[df['club_name'] == TIME_ESCOLHIDO].copy()

# 2. Adicionar Posição Agregada para o filtro
mercado_completo['posicao_agregada'] = mercado_completo['main_position'].map(mapa_posicoes)
mercado_completo = mercado_completo.dropna(subset=['posicao_agregada'])

# 3. Aplicar Filtro Top-K por Posição
mercado_filtrado_ids = set()
for pos in requisitos_posicao.keys():
    candidatos_pos = mercado_completo[
        (mercado_completo['posicao_agregada'] == pos) &
        (mercado_completo['value_eur'] <= VALOR_MAX_JOGADOR)
    ].nlargest(K_MERCADO_POR_POS, 'overall_rating')
    
    mercado_filtrado_ids.update(candidatos_pos.index)

# 'mercado' é o dataframe final podado que usaremos
mercado = mercado_completo.loc[list(mercado_filtrado_ids)].copy()

print(f"Mercado reduzido de {len(mercado_completo)} para {len(mercado)} jogadores.")

# 4. Calcular Orçamento Salarial Dinâmico
salario_anual_atual_t0 = elenco_atual['wage_eur'].sum() * 52
WAGE_BUDGET_YEAR = salario_anual_atual_t0 * (1 + PERCENTUAL_FOLGA_SALARIAL)

print(f"\nSalário Anual Atual (t=0): €{salario_anual_atual_t0:,.0f}")
print(f"Teto Salarial Definido (Atual + {PERCENTUAL_FOLGA_SALARIAL:.0%}): €{WAGE_BUDGET_YEAR:,.0f}")

# 5. Definir Tamanho Máximo do Elenco
TAMANHO_MAX_ELENCO = 30

Iniciando Poda 1: Filtrando mercado (Top-K por Posição)...
Mercado reduzido de 18304 para 1600 jogadores.

Salário Anual Atual (t=0): €63,440,000
Teto Salarial Definido (Atual + 20%): €76,128,000


In [61]:
# --- [PIPELINE ETAPA 2: CRIAÇÃO DOS DICIONÁRIOS DE DADOS] ---
# (Lógica da sua Célula [11] original)

# Dicionários de dados separados
elenco_data = elenco_atual.to_dict('index')
players_data = mercado.to_dict('index') # 'players_data' é só o mercado

# --- IDs GLOBAIS ---
elenco_ids = elenco_atual.index.tolist()
mercado_ids = mercado.index.tolist() # IDs do mercado
todos_os_ids = elenco_ids + mercado_ids # IDs de todos os jogadores (atuais + mercado)

# Dicionário de dados COMPLETO (necessário para 'titular_vars')
players_data_completo = {}
players_data_completo.update(elenco_data)
players_data_completo.update(players_data)

print(f"IDs definidos: {len(elenco_ids)} jogadores no elenco, {len(mercado_ids)} no mercado.")
print(f"Total de {len(todos_os_ids)} jogadores no universo.")

IDs definidos: 28 jogadores no elenco, 1600 no mercado.
Total de 1628 jogadores no universo.


In [62]:
# --- [PIPELINE ETAPA 3: EXECUÇÃO DA SIMULAÇÃO (ALMANAQUE)] ---
# (Lógica da sua Célula [12] original)

print("Iniciando o pré-cálculo da evolução dos jogadores...")
start_time = time.time()

# O "almanaque" final
dados_temporais = {}

for player_id, stats_t0 in players_data_completo.items():
    dados_temporais[player_id] = {}
    dados_temporais[player_id][0] = stats_t0
    
    for t in range(1, NUM_JANELAS):
        stats_anterior = dados_temporais[player_id][t-1]
        try:
            novas_stats = evoluir_jogador_uma_janela(stats_anterior, t)
        except Exception as e:
            print(f"Erro ao processar jogador {stats_anterior.get('name', player_id)}: {e}")
            novas_stats = stats_anterior
            
        dados_temporais[player_id][t] = novas_stats

end_time = time.time()
print(f"Pré-cálculo concluído em {end_time - start_time:.2f} segundos.")
print(f"Total de {len(dados_temporais)} jogadores processados para {NUM_JANELAS} janelas.")

# --- Verificação da Evolução ---
print("\n--- Verificando a evolução de um jogador (ex: Wallace Yan) ---")
try:
    JOGADOR_ID_EXEMPLO = [pid for pid, data in elenco_data.items() if data['name'] == 'Wallace Yan'][0]
    for t in range(NUM_JANELAS):
        stats = dados_temporais[JOGADOR_ID_EXEMPLO][t]
        print(f"Janela {t} (Idade {stats['age']}): OVR: {stats['overall_rating']}, Pot: {stats['potential']}, Valor: €{stats['value_eur']:,}")
except Exception:
    print("Não foi possível encontrar 'Wallace Yan' no elenco_data para o exemplo.")

Iniciando o pré-cálculo da evolução dos jogadores...
Pré-cálculo concluído em 0.03 segundos.
Total de 1628 jogadores processados para 4 janelas.

--- Verificando a evolução de um jogador (ex: Wallace Yan) ---
Janela 0 (Idade 19): OVR: 69, Pot: 82, Valor: €3,300,000.0
Janela 1 (Idade 19): OVR: 70, Pot: 82, Valor: €4,367,000.0
Janela 2 (Idade 20): OVR: 72, Pot: 82, Valor: €6,493,000.0
Janela 3 (Idade 20): OVR: 72, Pot: 82, Valor: €7,584,000.0


In [63]:
# --- [PIPELINE ETAPA 4: PODA DA QUÍMICA (PARES RELEVANTES)] ---
# (Lógica da sua Célula [13] original)

print("Iniciando pré-cálculo dos Titulares t=0 e Pares de Química Relevantes...")
start_time_precalc = time.time()

# 1. Definir o Time Titular de t=0 (Greedy)
dados_elenco_t0 = {pid: dados_temporais[pid][0] for pid in elenco_ids}
df_elenco_t0 = pd.DataFrame.from_dict(dados_elenco_t0, orient='index')
df_elenco_t0['posicao_agregada'] = df_elenco_t0['main_position'].map(mapa_posicoes)

titulares_t0_ids = []
for pos_form, num_necessarios in formacao_titular.items():
    candidatos = df_elenco_t0[df_elenco_t0['posicao_agregada'] == pos_form]
    candidatos = candidatos[~candidatos.index.isin(titulares_t0_ids)]
    melhores_candidatos = candidatos.nlargest(num_necessarios, 'overall_rating')
    titulares_t0_ids.extend(melhores_candidatos.index.tolist())

titulares_t0_set = set(titulares_t0_ids)
print(f"Time Titular de t=0 (Greedy) definido com {len(titulares_t0_set)} jogadores.")

# 2. PODA 2: Definir Pares de Química Relevantes
mercado['posicao_agregada'] = mercado['main_position'].map(mapa_posicoes)
mercado_por_pos = {
    pos: mercado[mercado['posicao_agregada'] == pos] 
    for pos in requisitos_posicao.keys()
}
pares_relevantes = set()

# Adiciona Pares (Elenco, Elenco)
for i in elenco_ids:
    for j in elenco_ids:
        if i < j:
            pares_relevantes.add((i, j))
print(f"Adicionados {len(pares_relevantes)} pares (Elenco-Elenco).")

# Adiciona Pares (Elenco, Mercado) - Poda Inteligente
pares_elenco_mercado_count = 0
for i in elenco_ids:
    pos_i = df_elenco_t0.loc[i]['posicao_agregada']
    posicoes_parceiras = {p_j for (p_i, p_j) in pares_taticos if p_i == pos_i}
    
    for pos_j in posicoes_parceiras:
        candidatos_parceiros = mercado_por_pos.get(pos_j)
        if candidatos_parceiros is None or candidatos_parceiros.empty:
            continue
            
        top_k_parceiros = candidatos_parceiros.nlargest(K_PARES_POR_JOGADOR, 'overall_rating')
        
        for j in top_k_parceiros.index:
            par = (min(i, j), max(i, j))
            pares_relevantes.add(par)
            pares_elenco_mercado_count += 1

print(f"Adicionados {pares_elenco_mercado_count} pares (Elenco-Mercado) taticamente relevantes.")
print(f"Total de {len(pares_relevantes)} pares de química (em vez de ~160k).")

# Limpa memória
del df_elenco_t0, candidatos, melhores_candidatos, mercado_por_pos

Iniciando pré-cálculo dos Titulares t=0 e Pares de Química Relevantes...
Time Titular de t=0 (Greedy) definido com 11 jogadores.
Adicionados 378 pares (Elenco-Elenco).
Adicionados 1620 pares (Elenco-Mercado) taticamente relevantes.
Total de 1998 pares de química (em vez de ~160k).


In [64]:
# --- [PIPELINE ETAPA 5: CONSTRUÇÃO DO MODELO (PULP)] ---
# (Lógica da sua Célula [14] original, lendo da Célula de Config)

print("Iniciando a construção do modelo v_Estoque (Otimizado)...")
start_time_build = time.time()

# 1. Parâmetros de Química (lidos da Célula de Config)
# --- CORREÇÃO: Define a lista JANELAS a partir do config ---
JANELAS = list(range(NUM_JANELAS)) 

S_MAX = GAIN_TITULARES / (1 - DECAY_QUIMICA) if (1 - DECAY_QUIMICA) != 0 else GAIN_TITULARES * 10 

# 2. Inicialização do Modelo
model = pulp.LpProblem("Planejamento_Estrategico_Elenco_v_Estoque", pulp.LpMaximize)

# 3. Variáveis de Decisão
var_indices = [(i, t) for i in todos_os_ids for t in JANELAS] # <--- Agora 'JANELAS' existe
var_indices_mercado = [(j, t) for j in mercado_ids for t in JANELAS]

no_elenco_vars = pulp.LpVariable.dicts("NoElenco", var_indices, cat='Binary')
contratar_vars = pulp.LpVariable.dicts("Contratar", var_indices_mercado, cat='Binary')
vender_vars = pulp.LpVariable.dicts("Vender", var_indices, cat='Binary')
titular_vars = pulp.LpVariable.dicts("Titular", var_indices, cat='Binary')

# 4. Variáveis de Química (Apenas para pares relevantes)
var_indices_pares = [(i, j, t) for (i, j) in pares_relevantes for t in JANELAS]
ParTitular = pulp.LpVariable.dicts("ParTitular", var_indices_pares, cat='Binary')
Quimica = pulp.LpVariable.dicts("Quimica", var_indices_pares, lowBound=0, upBound=S_MAX, cat='Continuous')

# 5. Variáveis de Orçamento
orcamento_transfer = pulp.LpVariable.dicts("OrcamentoTransfer", JANELAS, lowBound=0)
orcamento_salario_disponivel = pulp.LpVariable.dicts("OrcamentoSalarial", JANELAS, lowBound=0)
print("Definição de variáveis concluída.")

# --- 6. Restrições ---
print("Adicionando restrições básicas...")
# A. Restrições Iniciais (t=0)
model += orcamento_transfer[0] == TRANSFER_BUDGET_INICIAL, "Orcamento_Inicial_Transfer"
model += orcamento_salario_disponivel[0] == WAGE_BUDGET_YEAR, "Orcamento_Inicial_Salario"

for i in todos_os_ids:
    model += no_elenco_vars[i, 0] == (1 if i in elenco_ids else 0), f"Inicio_Elenco_{i}"
    model += titular_vars[i, 0] == (1 if i in titulares_t0_set else 0), f"Inicio_Titular_{i}"

# B/C. Restrições de Fluxo e Operação (t > 0)
for t in JANELAS:
    if t > 0: 
        # Fluxo de Jogadores
        for i in todos_os_ids:
            if i in mercado_ids:
                 model += no_elenco_vars[i, t] == no_elenco_vars[i, t-1] - vender_vars[i, t-1] + contratar_vars[i, t-1], f"Fluxo_Elenco_Mercado_{i}_{t}"
            else: 
                 model += no_elenco_vars[i, t] == no_elenco_vars[i, t-1] - vender_vars[i, t-1], f"Fluxo_Elenco_Original_{i}_{t}"

        # Fluxo de Orçamento
        gastos_transfer_t_menos_1 = pulp.lpSum([dados_temporais[j][t-1]['value_eur'] * contratar_vars[j, t-1] for j in mercado_ids])
        receitas_vendas_t_menos_1 = pulp.lpSum([dados_temporais[i][t-1]['value_eur'] * vender_vars[i, t-1] for i in todos_os_ids])
        model += orcamento_transfer[t] == orcamento_transfer[t-1] - gastos_transfer_t_menos_1 + receitas_vendas_t_menos_1, f"Fluxo_Orcamento_Transfer_{t}"

        # Teto Salarial
        salario_total_t = pulp.lpSum([dados_temporais[i][t]['wage_eur']*52 * no_elenco_vars[i, t] for i in todos_os_ids])
        model += salario_total_t <= WAGE_BUDGET_YEAR, f"Teto_Salarial_{t}"
        
        # Formação Titular
        for pos_formacao, num_necessarios in formacao_titular.items():
            jogadores_da_posicao = [i for i in todos_os_ids if mapa_posicoes.get(dados_temporais[i][t]['main_position']) == pos_formacao]
            model += pulp.lpSum([titular_vars[i, t] for i in jogadores_da_posicao]) == num_necessarios, f'Titulares_{pos_formacao}_{t}'
            
        # Profundidade do Elenco
        for pos_agregada, min_req in requisitos_posicao.items():
            jogadores_da_posicao = [i for i in todos_os_ids if mapa_posicoes.get(dados_temporais[i][t]['main_position']) == pos_agregada]
            model += pulp.lpSum([no_elenco_vars[i, t] for i in jogadores_da_posicao]) >= min_req, f"Restricao_Min_{pos_agregada}_{t}"

        # Tamanho Máximo do Elenco
        model += pulp.lpSum([no_elenco_vars[i, t] for i in todos_os_ids]) <= TAMANHO_MAX_ELENCO, f"Tamanho_Max_Elenco_{t}"
        
    # Restrições de Ligação (Todas as janelas)
    for i in todos_os_ids:
        model += vender_vars[i, t] <= no_elenco_vars[i, t], f"Ligacao_Venda_{i}_{t}"
        model += titular_vars[i, t] <= no_elenco_vars[i, t], f"Ligacao_Titular_{i}_{t}"
    
    # Restrição de Compra Única (Definida uma vez)
    if t == 0:
        for j in mercado_ids:
             model += pulp.lpSum([contratar_vars[j, t_futuro] for t_futuro in JANELAS]) <= 1, f"Compra_Unica_{j}"

# --- 7. Restrições de Química (Apenas para Pares Relevantes) ---
print("Adicionando restrições de química (otimizado)...")
for (i, j) in pares_relevantes:
    
    model += Quimica[(i, j, 0)] == 0, f"Stock_Init_{i}_{j}" # Inicialização do Estoque

    for t in JANELAS:
        # Linearização do ParTitular
        model += ParTitular[(i, j, t)] <= titular_vars[i, t], f"Linear_Par_1_{i}_{j}_{t}"
        model += ParTitular[(i, j, t)] <= titular_vars[j, t], f"Linear_Par_2_{i}_{j}_{t}"
        model += ParTitular[(i, j, t)] >= titular_vars[i, t] + titular_vars[j, t] - 1, f"Linear_Par_3_{i}_{j}_{t}"

        # Dinâmica do Estoque (t > 0)
        if t > 0:
            model += Quimica[(i, j, t)] == DECAY_QUIMICA * Quimica[(i, j, t-1)] + GAIN_TITULARES * ParTitular[(i, j, t-1)], f"Stock_Dyn_{i}_{j}_{t}"
print("Adição de restrições de química concluída.")

# --- 8. Função Objetivo ---
# (Lendo os pesos da Célula de Config)
PESO_TITULAR = 1.0
PESO_RESERVA = 0.15

obj_total = [] 
bonus_quimica_total = [] 

for t in JANELAS:
    if t > 0: # Score é contado para t=1, 2, 3
        
        # Score Base (OVR, Potencial, Físico)
        obj_elenco_t = pulp.lpSum(
            (dados_temporais[i][t]['overall_rating'] * w_qualidade +
             dados_temporais[i][t].get('growth_potential', 0) * w_potencial +
             dados_temporais[i][t]['physical'] * w_fisico) *
            (PESO_RESERVA * no_elenco_vars[i, t] + (PESO_TITULAR - PESO_RESERVA) * titular_vars[i, t])
            for i in todos_os_ids
        )
        obj_total.append(obj_elenco_t)
        
        # Bônus de Entrosamento (Estoque S)
        for (i, j) in pares_relevantes:
            bonus_quimica_total.append(BONUS_ENTROSAMENTO * Quimica[(i, j, t)])

model += pulp.lpSum(obj_total) + pulp.lpSum(bonus_quimica_total), "Score_Acumulado_Com_Entrosamento"

end_time_build = time.time()
print(f"\nConstrução do modelo concluída em {end_time_build - start_time_build:.2f} segundos.")

Iniciando a construção do modelo v_Estoque (Otimizado)...
Definição de variáveis concluída.
Adicionando restrições básicas...
Adicionando restrições de química (otimizado)...
Adição de restrições de química concluída.

Construção do modelo concluída em 0.67 segundos.


In [65]:
# --- [PIPELINE ETAPA 6: RESOLUÇÃO E ANÁLISE] ---
# (Lógica da sua Célula [15] original, lendo da Célula de Config)

print("Iniciando a resolução do modelo dinâmico...")
print(f"Parâmetros: Limite de Tempo={LIMITE_TEMPO}s, Gap de Otimização={GAP_RELATIVO*100}%\n")
start_time_solve = time.time()

# 1. Parâmetros do Solver (lidos da Célula de Config)
solver_params = {"gapRel": GAP_RELATIVO, "timeLimit": LIMITE_TEMPO}

model.solve(pulp.PULP_CBC_CMD(**solver_params))

end_time_solve = time.time()
print(f"\nResolução concluída em {end_time_solve - start_time_solve:.2f} segundos.")

# --- 2. Exibição dos Resultados ---
print(f"\nStatus da Solução: {pulp.LpStatus[model.status]}\n")

if pulp.LpStatus[model.status] in ['Optimal', 'Undefined']: # 'Undefined' é retornado se o timeLimit é atingido
    
    if pulp.LpStatus[model.status] == 'Undefined':
        print("ATENÇÃO: O limite de tempo foi atingido. A solução abaixo é a MELHOR ENCONTRADA, mas pode não ser a ÓTIMA.")
    
    print("--- PLANO ESTRATÉGICO DE 2 ANOS (4 JANELAS) ---")
    
    for t in JANELAS:
        print("\n" + "="*30)
        print(f"         JANELA {t} (Temporada {2024 + t//2} - {2025 + t//2})")
        print("="*30)
        
        # --- A. Decisões de Transferência ---
        print("\n  A. Decisões de Transferência:")
        contratados_t = []
        for j in mercado_ids:
            if contratar_vars[j, t].varValue > 0.9: # Mais robusto para erros de ponto flutuante
                stats = dados_temporais[j][t]
                contratados_t.append(stats)
                print(f"    [CONTRATAÇÃO] {stats['name']} (OVR: {stats['overall_rating']}) por €{stats['value_eur']:,.0f}")
        
        vendidos_t = []
        for i in todos_os_ids:
            if vender_vars[i, t].varValue > 0.9:
                stats = dados_temporais[i][t]
                vendidos_t.append(stats)
                print(f"    [VENDA]         {stats['name']} (OVR: {stats['overall_rating']}) por €{stats['value_eur']:,.0f}")
        
        if not contratados_t and not vendidos_t:
            print("    Nenhuma transação nesta janela.")

        # --- B. Time Titular da Temporada ---
        print("\n  B. Time Titular da Temporada:")
        time_titular_t = []
        for i in todos_os_ids:
            if titular_vars[i, t].varValue > 0.9:
                time_titular_t.append(dados_temporais[i][t])
        
        mapa_ordem = {pos: idx for idx, pos in enumerate(formacao_titular.keys())}
        time_titular_ordenado_t = sorted(
            time_titular_t, 
            key=lambda x: mapa_ordem.get(mapa_posicoes.get(x['main_position']), 99)
        )
        
        if t == 0:
            print("    (Este é o time titular inicial 'Greedy' definido no pré-cálculo)")
        
        for jogador in time_titular_ordenado_t:
            print(f"    - {jogador['main_position']:<4} | {jogador['name']:<25} | (Idade: {jogador['age']}, OVR: {jogador['overall_rating']})")

        # --- C. Status do Elenco e Finanças ---
        print("\n  C. Status do Elenco e Finanças (Final da Janela):")
        elenco_total_t = [
            dados_temporais[i][t] for i in todos_os_ids if no_elenco_vars[i, t].varValue > 0.9
        ]
        salario_total_t = sum(p['wage_eur'] for p in elenco_total_t) * 52
                
        print(f"    - Tamanho do Elenco: {len(elenco_total_t)} jogadores")
        
        if t + 1 < NUM_JANELAS:
            print(f"    - Salário Anual: €{salario_total_t:,.0f} (Teto: €{WAGE_BUDGET_YEAR:,.0f})")
            print(f"    - Orçamento p/ Próxima Janela (t={t+1}): €{orcamento_transfer[t+1].varValue:,.0f}")
        else:
            print(f"    - Salário Anual Final: €{salario_total_t:,.0f}")
            print(f"    - Orçamento Final (t={t}): €{orcamento_transfer[t].varValue:,.0f}")

elif pulp.LpStatus[model.status] == 'Infeasible':
    print("O modelo não encontrou uma solução (INFEASIBLE).")
    print("\n--- DICAS PARA DEBUGAR (INFEASIBLE) ---")
    print("1. O Orçamento/Teto Salarial estão muito baixos?")
    print("2. As Restrições de Posição são muito rígidas? (ex: 'requisitos_posicao')")
    print("3. O Mercado (Poda K_MERCADO_POR_POS) está muito pequeno? O solver não encontrou jogadores suficientes para todas as posições.")
else:
    print(f"O modelo falhou com o status: {pulp.LpStatus[model.status]}")

Iniciando a resolução do modelo dinâmico...
Parâmetros: Limite de Tempo=1800s, Gap de Otimização=1.0%

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/danielbarros/projects/Otimizacao_Trab/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/3167bb253b3d401ea547e468d10f5553-pulp.mps -max -sec 1800 -ratio 0.01 -timeMode elapsed -branch -printingOptions all -solution /var/folders/k8/xzk1z07s6c58p_hh_nnx6nk40000gn/T/3167bb253b3d401ea547e468d10f5553-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54796 COLUMNS
At line 298600 RHS
At line 353392 BOUNDS
At line 395313 ENDATA
Problem MODEL has 54791 rows, 41925 columns and 160185 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 1800
ratioGap was changed from 0 to 0.01
Option for timeMode changed from cpu to elapsed
Continuous objective value is 155804 - 1.06 seco